In [11]:
import cv2
import numpy as np
import pandas as pd
import os

# Define la ruta base donde se encuentran las imágenes
ruta_base_imagenes = 'img/'
archivo='Metrica_8_bits_9_dim_Tsallis.xlsx'
#RSA_umbral	HBA_umbral	OPA_umbral	BES_umbral	GWO_umbral	CSA_umbral	HHO_umbral	TSO_umbral
# Nombre del algoritmo array
Bioinspirado = ['Ejecucion','RSA_umbral','HBA_umbral','OPA_umbral','BES_umbral','GWO_umbral','CSA_umbral','HHO_umbral','TSO_umbral']    

algoritmo = Bioinspirado[5]  # Nombre del algoritmo

# Cargar el archivo de Excel
df = pd.read_excel(archivo, usecols=[Bioinspirado[0],'Imagen', algoritmo])
df = df[df[Bioinspirado[0]] == 0]
# Limpiar y extraer los nombres de las imágenes y los umbrales
#df['Imagen'] = df['Imagen'].str.extract(r'\]\s*(.*)')  # Elimina el corchete y el número al principio
umbrales = df[algoritmo].apply(lambda x: eval(x))  # Convierte el string de lista a una lista real

# Función para aplicar los umbrales a la imagen
def aplicar_umbrales(imagen, umbrales, semilla=42):
    # Establecer la semilla para generar colores pseudoaleatorios
    np.random.seed(semilla)
    
    # Suponiendo que la imagen está en escala de grises
    imagen_color = cv2.cvtColor(imagen, cv2.COLOR_GRAY2BGR)
    
    for i, umbral in enumerate(umbrales):
        _, img_umbral = cv2.threshold(imagen, umbral, 255, cv2.THRESH_BINARY)
        # Generar un color único a partir de la semilla
        color = tuple(np.random.randint(0, 255, 3).tolist())
        imagen_color[img_umbral == 255] = color
    
    return imagen_color

# Procesar cada imagen con sus umbrales
for index, row in df.iterrows():
    nombre_imagen = row['Imagen']
    umbrales_img = umbrales[index]
    ruta_completa = os.path.join(ruta_base_imagenes, nombre_imagen)
    
    # Cargar la imagen
    imagen = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
    if imagen is not None:
        imagen_procesada = aplicar_umbrales(imagen, umbrales_img)
        # Guardar o mostrar la imagen procesada
        cv2.imwrite(f'destino/{nombre_imagen}', imagen_procesada)
    else:
        print(f"No se pudo cargar la imagen: {nombre_imagen}")

In [12]:
import cv2
import numpy as np
import pandas as pd
import os

# Define la ruta base donde se guardaron las imágenes procesadas
ruta_base_destino = 'destino/'
# Lista para almacenar las imágenes cargadas
imagenes = []

# Tamaño objetivo para cada imagen
altura_objetivo, ancho_objetivo = 300, 300

# Leer y redimensionar las imágenes en el orden correcto
for nombre_imagen in df['Imagen']:
    ruta_completa = os.path.join(ruta_base_destino, nombre_imagen)
    imagen = cv2.imread(ruta_completa)
    if imagen is not None:
        # Redimensionar la imagen
        imagen_redimensionada = cv2.resize(imagen, (ancho_objetivo, altura_objetivo))
        imagenes.append(imagen_redimensionada)
    else:
        print(f"No se pudo cargar la imagen: {nombre_imagen}")

# Verificar si se cargaron imágenes
if len(imagenes) > 0:
    # Apilar todas las imágenes para formar una sola imagen de columna
    imagen_matriz = np.vstack(imagenes)
    
    # Guardar la imagen matriz
    cv2.imwrite('imagen_matriz.png', imagen_matriz)
else:
    print("No se encontraron imágenes para crear la matriz.")